In [6]:
import os
import platform

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '3,4'
data_dir = './data'
output_dir = './output/my_model'

print(platform.node())
print('data_dir: ', data_dir)
print('output_dir: ', output_dir)

bld
data_dir:  ./data
output_dir:  ./output/my_model


In [7]:
from utils import prepare_data, seed_everything

train_dataset, val_dataset, test_dataset, _ = prepare_data(data_dir)
y_batch = next(iter(train_dataset))[0]

In [8]:
from models import SpecNovelCNN

seed_everything(0)

model = SpecNovelCNN()
model(y_batch)  # build the model by passing a sample input
model.summary(expand_nested=True)

Model: "spec_novel_cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block0 (Sequential)         (256, 256, 32)            3232      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| conv1 (Conv1D)             (256, 512, 32)            128      |
|                                                               |
| conv2 (Conv1D)             (256, 512, 32)            3104     |
|                                                               |
| pool (AveragePooling1D)    (256, 256, 32)            0        |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 block1 (Sequential)         (256, 128, 64)            18560     
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| conv1 (Conv1D)             (256, 256, 64)            6208     |
|                                                               |
| conv2 (Conv1D)             (256, 256, 64)       

In [9]:
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger

from metrics import CC, RRMSE, SNR

model.compile(
    optimizer=Adam(learning_rate=5e-5, epsilon=1e-8),
    loss=MeanSquaredError(),
    metrics=[CC(), RRMSE(), SNR()],
)

callbacks = [
    EarlyStopping(
        monitor='val_snr',
        mode='max',
        patience=20,
        restore_best_weights=True,
        verbose=1,
    ),
    ModelCheckpoint(
        filepath=f'{output_dir}/my_model.h5',
        mode='max',
        monitor='val_snr',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir=output_dir),
    CSVLogger(f'{output_dir}/my_model.csv'),
]

history = model.fit(
    train_dataset,
    epochs=200,
    steps_per_epoch=200,
    validation_data=val_dataset,
    callbacks=callbacks,
)

Epoch 1/200


2024-01-22 07:24:02.788591: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inspec_novel_cnn_1/spec_block0/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


200/200 [==============================] - 14s 37ms/step - loss: 0.5975 - cc: 0.0684 - rrmse: 0.9980 - snr: 0.0182 - val_loss: 0.5036 - val_cc: 0.0817 - val_rrmse: 0.9975 - val_snr: 0.0222
Epoch 2/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4944 - cc: 0.2168 - rrmse: 0.9745 - snr: 0.2304 - val_loss: 0.4696 - val_cc: 0.2711 - val_rrmse: 0.9626 - val_snr: 0.3370
Epoch 3/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4573 - cc: 0.3175 - rrmse: 0.9487 - snr: 0.4683 - val_loss: 0.4411 - val_cc: 0.3659 - val_rrmse: 0.9321 - val_snr: 0.6204
Epoch 4/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4380 - cc: 0.3807 - rrmse: 0.9276 - snr: 0.6695 - val_loss: 0.4240 - val_cc: 0.4062 - val_rrmse: 0.9136 - val_snr: 0.7999
Epoch 5/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4239 - cc: 0.3948 - rrmse: 0.9196 - snr: 0.7482 - val_loss: 0.4096 - val_cc: 0.4340 - val_rrmse: 0.8992 - val_snr: 0.9443
Epoch 6/200

In [10]:
model.evaluate(test_dataset)

22/22 [==============================] - 0s 14ms/step - loss: 0.0794 - cc: 0.9235 - rrmse: 0.3382 - snr: 10.9835


[0.07940466701984406,
 0.9234558343887329,
 0.3382062315940857,
 10.983481407165527]